## Generate Hyp3 interferogram stack

This notebook will queue up processing jobs in Hyp3 and then download and prepare them for use in Mintpy. Code adapted from [Hyp3 tutorial](https://nbviewer.org/github/ASFHyP3/hyp3-docs/blob/main/docs/tutorials/hyp3_insar_stack_for_ts_analysis.ipynb).


In [5]:
site = 'NorthSlopeEastD102'
year = 2025

### Prepare environment

Import functions and load metadata for processing. The environment requires several packages from the Alaska Satellite Facility to function, specifically [asf_search](https://docs.asf.alaska.edu/asf_search/basics/) and [hyp3_sdk](https://hyp3-docs.asf.alaska.edu/using/sdk/).

In [6]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
from dateutil.parser import parse as parse_date
from typing import List, Union
from osgeo import gdal
from solid_utils import permafrost_utils as pu
import asf_search as asf
import hyp3_sdk as sdk

In [7]:
################# Set Directories ##########################################
# print('\nCurrent directory:',os.getcwd())

# if 'work_dir' not in locals():
    # work_dir = Path.cwd()/'work'/'permafrost_ouputs'/site/str(year)
# work_dir = Path(f'/scratch/nisar-st-calval-solidearth/S1_permafrost/{site}/{str(year)}')
work_dir = Path(f'/scratch/nisar-st-calval-solidearth/permafrost/Hyp3_S1/{site}/{str(year)}')

print("Work directory:", work_dir)
work_dir.mkdir(parents=True, exist_ok=True)
# Change to Workdir   
os.chdir(work_dir)
       
hyp3_dir = work_dir/'products'               #aka gunwdir
hyp3_dir.mkdir(parents=True, exist_ok=True)
print("   Hyp3  dir:", hyp3_dir) 
    
mintpy_dir = work_dir/'MintPy' 
mintpy_dir.mkdir(parents=True, exist_ok=True)
print("   MintPy  dir:", mintpy_dir)
############################################################################
### List of CalVal Sites:
'''
Set NISAR calval sites:
    NorthSlopeEastD102      : North Slope of Alaska including Dalton highway, Sentinel-1 descending path (track) 102. 
                            : The field validation sites are located in this frame.
    NorthSlopeWestD44       : Western area of the North Slope of Alaska, Sentinel-1 descending path (track) 44.
    NorthwestTerritoriesA78 : North east of Yellowknife, Sentinel-1 ascending path (track) 78.


Hyp3 & MintPy parameters:
    calval_location    : name
    region_identifier  : WTK string with latlon point for identifying image in Hyp3
    subset_region      : subset analysis area, in UTM. Given in '[ymin:ymax,xmin:xmax]' or 'none'
    download_start_date : download start date as YYYMMDD  
    download_end_date   : download end date as YYYMMDD
    mintpy_ref_loc     : reference point for use in mintpy in UTM Y,X order. Projection must be that of the Hyp3 images.
    tempBaseMax        : maximum number of days, don't use interferograms longer than this value 
    ifgExcludeList     : default is not to exclude any interferograms
    maskWater          : interior locations don't need to mask water
    sentinel_path      : asfPath number for identifying image. Also called track.
    sentinel_frame     : asfFrame number for identifying image
'''
sites = {
    ##########  NORTH SLOPE EAST (DALTON/TOOLIK) ##############
    'NorthSlopeEastD102' : {'calval_location' : 'NorthSlopeEastD102',
            'region_identifier' : 'POINT(-149.37 69.09)',
            'subset_region' : '[7620213:7686754, 641941:679925]',            
            'download_start_date' : '20230525',
            'download_end_date'   : '20230910',
            'mintpy_ref_loc'      : '7651392, 666923',
            'tempBaseMax' : '36',
            'ifgExcludeList' : 'auto',
            'maskWater' : 'True',
            'sentinel_direction' : 'DESCENDING',
            'sentinel_path' : '102',
            'sentinel_frame' : '362'},
    'NorthSlopeWestD44' : {'calval_location' : 'NorthSlopeWestD44',
            'region_identifier' : 'POINT(-160.25 68.82)',
            'subset_region' : 'none',
            'download_start_date' : '20230525',
            'download_end_date'   : '20230910',
            'mintpy_ref_loc' : '7684081, 437118',
            'tempBaseMax' : '36',
            'ifgExcludeList' : 'auto',
            'maskWater' : 'True',
            'sentinel_direction' : 'DESCENDING',
            'sentinel_path' : '44',
            'sentinel_frame' : '362'},
    'NorthwestTerritoriesA78' : {'calval_location' : 'NorthwestTerritoriesA78',
            'region_identifier' : 'POINT(-111.35 63.20)',
            'subset_region' : 'none',
            'download_start_date' : '20230525',
            'download_end_date'   : '20230910',
            'mintpy_ref_loc'      : '6980804, 522414',
            'tempBaseMax' : '36',
            'ifgExcludeList' : 'auto',
            'maskWater' : 'True',
            'sentinel_direction' : 'ASCENDING',
            'sentinel_path' : '78',
            'sentinel_frame' : '204'},    
}

Work directory: /scratch/nisar-st-calval-solidearth/permafrost/Hyp3_S1/NorthSlopeEastD102/2025
   Hyp3  dir: /scratch/nisar-st-calval-solidearth/permafrost/Hyp3_S1/NorthSlopeEastD102/2025/products
   MintPy  dir: /scratch/nisar-st-calval-solidearth/permafrost/Hyp3_S1/NorthSlopeEastD102/2025/MintPy


### Generate SBAS image pairs

Identify stack of SLC images and determine SBAS pairs.

In [8]:
lonlat = sites[site]['region_identifier']
frame = sites[site]['sentinel_frame']
if sites[site]['sentinel_direction']=='ASCENDING':
    direction = asf.constants.ASCENDING
elif sites[site]['sentinel_direction']=='DESCENDING':
    direction = asf.constants.DESCENDING

stack_start = parse_date(f'{year}-05-20 00:00:00Z')
stack_end = parse_date(f'{year}-09-20 00:00:00Z')

search_results = asf.geo_search(platform=asf.constants.SENTINEL1, 
                                intersectsWith=lonlat, 
                                start=stack_start,
                                end=stack_end,
                                processingLevel=asf.constants.SLC,
                                beamMode=asf.constants.IW,
                                flightDirection=direction,
                                asfFrame=int(frame))

In [10]:
baseline_results = asf.baseline_search.stack_from_product(search_results[-1])

columns = list(baseline_results[0].properties.keys()) + ['geometry', ]
data = [list(scene.properties.values()) + [scene.geometry, ] for scene in baseline_results]

stack = pd.DataFrame(data, columns=columns)
stack['startTime'] = stack.startTime.apply(parse_date)

# stack_start = parse_date(f'{year}-05-25 00:00:00Z')
# stack_end = parse_date(f'{year}-09-10 00:00:00Z')

stack = stack.loc[(stack_start <= stack.startTime) & (stack.startTime <= stack_end)]

stack

,centerLat,centerLon,stopTime,fileID,flightDirection,pathNumber,processingLevel,url,startTime,sceneName,...,processingDate,sensor,groupID,pgeVersion,fileName,beamModeType,s3Urls,temporalBaseline,perpendicularBaseline,geometry
239,68.9318,-151.7644,2025-05-23T16:44:24Z,S1A_IW_SLC__1SDV_20250523T164356_20250523T1644...,DESCENDING,102,SLC,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,2025-05-23 16:43:56+00:00,S1A_IW_SLC__1SDV_20250523T164356_20250523T1644...,...,2025-05-23T16:43:56Z,C-SAR,S1A_IWDV_0361_0368_059324_102,003.91,S1A_IW_SLC__1SDV_20250523T164356_20250523T1644...,IW,[s3://asf-ngap2w-p-s1-slc-7b420b89/S1A_IW_SLC_...,0,0.0,"{'coordinates': [[[-148.043823, 69.466782], [-..."
240,68.9321,-151.7638,2025-06-04T16:44:24Z,S1A_IW_SLC__1SDV_20250604T164356_20250604T1644...,DESCENDING,102,SLC,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,2025-06-04 16:43:56+00:00,S1A_IW_SLC__1SDV_20250604T164356_20250604T1644...,...,2025-06-04T16:43:56Z,C-SAR,S1A_IWDV_0361_0368_059499_102,003.91,S1A_IW_SLC__1SDV_20250604T164356_20250604T1644...,IW,[s3://asf-ngap2w-p-s1-slc-7b420b89/S1A_IW_SLC_...,12,-17.0,"{'coordinates': [[[-148.043106, 69.467117], [-..."
241,68.9322,-151.7644,2025-06-16T16:44:23Z,S1A_IW_SLC__1SDV_20250616T164355_20250616T1644...,DESCENDING,102,SLC,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,2025-06-16 16:43:55+00:00,S1A_IW_SLC__1SDV_20250616T164355_20250616T1644...,...,2025-06-16T16:43:55Z,C-SAR,S1A_IWDV_0361_0368_059674_102,003.92,S1A_IW_SLC__1SDV_20250616T164355_20250616T1644...,IW,[s3://asf-ngap2w-p-s1-slc-7b420b89/S1A_IW_SLC_...,24,6.0,"{'coordinates': [[[-148.0439, 69.46711], [-154..."
242,68.9321,-151.7664,2025-06-28T16:44:22Z,S1A_IW_SLC__1SDV_20250628T164355_20250628T1644...,DESCENDING,102,SLC,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,2025-06-28 16:43:55+00:00,S1A_IW_SLC__1SDV_20250628T164355_20250628T1644...,...,2025-06-28T16:43:55Z,C-SAR,S1A_IWDV_0361_0368_059849_102,003.92,S1A_IW_SLC__1SDV_20250628T164355_20250628T1644...,IW,[s3://asf-ngap2w-p-s1-slc-7b420b89/S1A_IW_SLC_...,36,57.0,"{'coordinates': [[[-148.045914, 69.467064], [-..."
243,68.9321,-151.7665,2025-07-10T16:44:22Z,S1A_IW_SLC__1SDV_20250710T164354_20250710T1644...,DESCENDING,102,SLC,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,2025-07-10 16:43:54+00:00,S1A_IW_SLC__1SDV_20250710T164354_20250710T1644...,...,2025-07-10T16:43:54Z,C-SAR,S1A_IWDV_0361_0368_060024_102,003.92,S1A_IW_SLC__1SDV_20250710T164354_20250710T1644...,IW,[s3://asf-ngap2w-p-s1-slc-7b420b89/S1A_IW_SLC_...,48,79.0,"{'coordinates': [[[-148.045944, 69.466995], [-..."
244,68.9319,-151.7635,2025-07-22T16:44:21Z,S1A_IW_SLC__1SDV_20250722T164353_20250722T1644...,DESCENDING,102,SLC,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,2025-07-22 16:43:53+00:00,S1A_IW_SLC__1SDV_20250722T164353_20250722T1644...,...,2025-07-22T16:43:53Z,C-SAR,S1A_IWDV_0361_0368_060199_102,003.92,S1A_IW_SLC__1SDV_20250722T164353_20250722T1644...,IW,[s3://asf-ngap2w-p-s1-slc-7b420b89/S1A_IW_SLC_...,60,-34.0,"{'coordinates': [[[-148.04303, 69.466797], [-1..."
245,68.9317,-151.7600,2025-08-03T16:44:21Z,S1A_IW_SLC__1SDV_20250803T164353_20250803T1644...,DESCENDING,102,SLC,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,2025-08-03 16:43:53+00:00,S1A_IW_SLC__1SDV_20250803T164353_20250803T1644...,...,2025-08-03T16:43:53Z,C-SAR,S1A_IWDV_0361_0368_060374_102,003.92,S1A_IW_SLC__1SDV_20250803T164353_20250803T1644...,IW,[s3://asf-ngap2w-p-s1-slc-7b420b89/S1A_IW_SLC_...,72,-152.0,"{'coordinates': [[[-148.039352, 69.466461], [-..."
246,68.9321,-151.7567,2025-08-15T16:44:21Z,S1A_IW_SLC__1SDV_20250815T164353_20250815T1644...,DESCENDING,102,SLC,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,2025-08-15 16:43:53+00:00,S1A_IW_SLC__1SDV_20250815T164353_20250815T1644...,...,2025-08-15T16:43:53Z,C-SAR,S1A_IWDV_0361_0368_060549_102,003.92,S1A_IW_SLC__1SDV_20250815T164353_20250815T1644...,IW,[s3://asf-ngap2w-p-s1-slc-7b420b89/S1A_IW_SLC_...,84,-256.0,"{'coordinates': [[[-148.035858, 69.466904], [-..."
247,68.9319,-151.75

In [11]:
sbas_pairs = set()

for reference, rt in stack.loc[::-1, ['sceneName', 'temporalBaseline']].itertuples(index=False):
    secondaries = stack.loc[
        (stack.sceneName != reference)
        & (stack.temporalBaseline - rt <= int(sites[site]['tempBaseMax']))
        & (stack.temporalBaseline - rt > 0)
    ]
    for secondary in secondaries.sceneName:
        sbas_pairs.add((reference, secondary))

### Do Hyp3 Processing

This step requires NASA Earthdata credentials. See [Hyp3 documentation](https://hyp3-docs.asf.alaska.edu/) for more information about using Hyp3 processing. Each user has an allotted Hyp3 processing quota, for which some of this processing is counted against.

In [11]:

hyp3 = sdk.HyP3(prompt=True)


NASA Earthdata Login username:  acjohnson16
NASA Earthdata Login password:  ········


In [23]:

jobs = sdk.Batch()
h3projname = f"{site}_NCV_{year}"
for reference, secondary in sbas_pairs:
    jobs += hyp3.submit_insar_job(reference, secondary, name=h3projname,
                                  include_dem=True, include_look_vectors=True)


This processing will likely take about an hour to complete. You cannot download data until the processing is finished. The following cell will check to see if the processing is still running.

In [24]:
# class stillrunningError(Error):
#     """raised when jobs are still running"""
#     pass

running = len(hyp3.find_jobs(status_code=["RUNNING"]))
pending = len(hyp3.find_jobs(status_code=["PENDING"]))

# if running > 0:
#     print('Jobs still running!')
#     # raise stillrunningError
if running+pending ==0:
    print('No jobs running, all clear')
else:
    print(f'Jobs still running!\n{pending} pending, {running} running')

No jobs running, all clear


### Download Data


h3projname = f"{site}_NCV_{year}"
batch = hyp3.find_jobs(name=h3projname)
insar_products = batch.download_files(hyp3_dir)
insar_products = [sdk.util.extract_zipped_product(ii) for ii in insar_products]


### Crop Interferograms

In [ ]:
# Crop to common overlap
files = hyp3_dir.glob('*/*_dem.tif')
overlap = pu.get_common_overlap(files)
pu.clip_hyp3_products_to_common_overlap(hyp3_dir, overlap)

## Generate MintPy Config file

In [ ]:
def writenumpyconfig(config_file,data_dir,reflalo = None, subset = None):
    """Write a mintpy config file.
    config_file: file name to write to
    data_dir: directory of .tif files
    reflalo (optional): 'y,x' of reference point in tif proj
    subset (optional): '[ymin:ymax,xmin:xmax]' in tif proj
    
    Note: hyp3 provides images in a utm projection, so lalo here refers to y,x in map coords"""
    
    cfgtext = f"""
    mintpy.load.processor        = hyp3
    ##---------interferogram datasets:
    mintpy.load.unwFile          = {data_dir}/*/*_unw_phase_clipped.tif
    mintpy.load.corFile          = {data_dir}/*/*_corr_clipped.tif
    ##---------geometry datasets:
    mintpy.load.demFile          = {data_dir}/*/*_dem_clipped.tif
    mintpy.load.incAngleFile     = {data_dir}/*/*_lv_theta_clipped.tif
    mintpy.load.azAngleFile      = {data_dir}/*/*_lv_phi_clipped.tif
    mintpy.load.waterMaskFile    = {data_dir}/*/*_water_mask_clipped.tif"""

    if reflalo:
            cfgtext+=f"""    
    mintpy.reference.lalo         = {reflalo}"""  #should be 'y,x' in map coords
    
    if not subset:
        subset = 'no'
    cfgtext+=f"""
    mintpy.subset.lalo            = {subset}""" #should be '[ymin:ymax,xmin:xmax]' in map coords

    mintpy_config = config_file
    mintpy_config.write_text(cfgtext)

In [13]:
config_file = mintpy_dir/f"{sites[site]['calval_location']}_{str(year)}.cfg"
writenumpyconfig(config_file,hyp3_dir,reflalo=sites[site]['mintpy_ref_loc'],subset = sites[site]['subset_region'])